In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
os.getcwd()

'/Users/chance/Desktop/LH_home/ksc'

In [20]:
folder = os.listdir()

file = []
for f in folder:
    if "실거래가" in f:
        data = pd.read_csv("./" + f, encoding="cp949", skiprows=15)
        data = data[list(data.columns[:11])]
        file.append(data)

gajwa = pd.concat(file)
gajwa = gajwa.dropna(axis=0)

rate = pd.read_csv("./지역별_전월세전환율_20221112193907.csv", encoding="cp949")
rate = rate.T
rate = rate[2:]

# 도로조건을 정수로 변환
road=[]
for a in gajwa["도로조건"]:
    if a=="-":
        a = "0"
    road.append(int(a.split("m")[0]))
gajwa["도로조건"] = road


# 년월 column 생성
year=[]
month=[]
for d in gajwa["계약년월"]:
    day = pd.to_datetime(d, format="%Y%m")
    year.append(day.year)
    month.append(day.month)

gajwa["year"] = year
gajwa["month"] = month

built_day = [pd.to_datetime(d, format="%Y").year for d in gajwa["건축년도"]]
gajwa["건축년도"] = built_day


#원핫인코딩으로 전월세 구분하기
gajwa = pd.get_dummies(gajwa, columns = ["전월세구분"])

# 계약년월 기준으로 전월세전환율 입력하기
month = []
for r in rate.index:
    new = "".join(r.split("."))
    month.append(int(new))

rate["계약년월"] = month
rate.columns = ["전월세전환율", "계약년월"]
rate.head()

gajwa = pd.merge(left=gajwa, right=rate, how="inner", on="계약년월")

keep = [int("".join(k.split(","))) for k in gajwa["보증금(만원)"]]
gajwa["보증금(만원)"] = keep


# 전월세전환율 기준으로 보증금+월세의 가치를 전세로 치환하여 value column으로 만들어준다.
value=[round(gajwa["월세(만원)"][i] * 12 / (gajwa["전월세전환율"][i]/100) + gajwa["보증금(만원)"][i]) for i in range(len(gajwa))]
gajwa["전세전환가격"] = value
gajwa["도로별 전세값평균"] = gajwa.groupby("도로명").전세전환가격.transform("mean")

gajwa

,시군구,번지,도로조건,계약면적(㎡),계약년월,계약일,보증금(만원),월세(만원),건축년도,도로명,year,month,전월세구분_월세,전월세구분_전세,전월세전환율,전세전환가격,도로별 전세값평균
0,경상남도 진주시 가좌동,1***,12,15.0,202107,4,150,30,2016,가좌길48번길,2021,7,1,0,6.4,5775,5537.686047
1,경상남도 진주시 가좌동,1***,12,19.0,202107,17,200,30,2019,가좌길48번길,2021,7,1,0,6.4,5825,5537.686047
2,경상남도 진주시 가좌동,1***,12,19.0,202107,18,5000,10,2019,가좌길48번길,2021,7,1,0,6.4,6875,5537.686047
3,경상남도 진주시 가좌동,1***,8,19.8,202107,17,200,32,2017,가좌길60번길,2021,7,1,0,6.4,6200,5763.877193
4,경상남도 진주시 가좌동,1***,12,20.0,202107,21,100,25,2015,가좌길48번길,2021,7,1,0,6.4,4788,5537.686047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1614,경상남도 진주시 가좌동,1***,8,30.0,201910,20,6000,0,2018,가좌길44번길,2019,10,0,1,7.0,6000,5922.176471
1615,경상남도 진주시 가좌동,4**,12,33.0,201910,20,1500,30,2010,개양로6번길,2019,10,1,0,7.0,6643,5982.984848
1616,경상남도 진주시 가좌동,5**,12,25.0,201904,5,300,33,2007,진주대로500번길,2019,4,1,0,6.0,6900,7203.802395
1617,경상남도 진주시 가좌동,4**,12,27.0,201904,16,250,30,2007,개양로6번길,2019,4,1,0,6.0,6250,5982.984848


In [21]:
gajwa_data = pd.DataFrame.drop(gajwa, columns=["시군구","계약년월", "도로명", "번지", "계약일", "보증금(만원)", "월세(만원)", "전월세전환율"], axis=1)

gajwa_data

,도로조건,계약면적(㎡),건축년도,year,month,전월세구분_월세,전월세구분_전세,전세전환가격,도로별 전세값평균
0,12,15.0,2016,2021,7,1,0,5775,5537.686047
1,12,19.0,2019,2021,7,1,0,5825,5537.686047
2,12,19.0,2019,2021,7,1,0,6875,5537.686047
3,8,19.8,2017,2021,7,1,0,6200,5763.877193
4,12,20.0,2015,2021,7,1,0,4788,5537.686047
...,...,...,...,...,...,...,...,...,...
1614,8,30.0,2018,2019,10,0,1,6000,5922.176471
1615,12,33.0,2010,2019,10,1,0,6643,5982.984848
1616,12,25.0,2007,2019,4,1,0,6900,7203.802395
1617,12,27.0,2007,2019,4,1,0,6250,5982.984848


# 머신러닝

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [23]:
X = gajwa_data.drop(["전세전환가격"], axis=1)
y = gajwa_data['전세전환가격']

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=1645)

In [25]:
X_train.shape

(1295, 8)

In [26]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [27]:
y_preds = lr.predict(X_test)
y_preds[0:5]

array([7370.1297574 , 4686.28881529, 5906.25588701, 3913.19228528,
       5611.41297675])

In [30]:
# qgis없는 회귀예측
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)

print(f"MSE : {mse:.3f}", f"RMSE: {rmse:.3f}")
print(f"variance score : {r2_score(y_test, y_preds):.3f}")

MSE : 1856441.454 RMSE: 1362.513
variance score : 0.613
